In [ ]:
from run_parallel import process_parallel_bold
import pandas as pd
import logging

# Silence pdfminer and pdfplumber logs
logging.getLogger("pdfminer").setLevel(logging.ERROR)
logging.getLogger("pdfplumber").setLevel(logging.ERROR)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:

df_all_granted, dic_questions = process_parallel_bold('data/granted/')
df_all_granted['final_visa_status'] = 'granted'

df_all_refused, dic_questions = process_parallel_bold('data/refused/',
                                field_order=list(dic_questions.keys()),
                                field_labels=dic_questions)
df_all_refused['final_visa_status'] = 'refused'

In [ ]:
print('df_all_granted',df_all_granted.shape)
print('df_all_refused',df_all_refused.shape)

df_all = pd.concat([df_all_granted,df_all_refused]).reset_index(drop=True)

print('df_all',df_all.shape)

In [ ]:
def postprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Post-procesa las celdas que contienen ';' de la siguiente manera:
      - Separa la cadena por ';'
      - Si el último segmento es un entero <100 y el penúltimo es 'si', 'no' o 'yes',
        devuelve el penúltimo.
      - En cualquier otro caso (incluyendo cuando no se cumplan esas condiciones),
        devuelve el valor original.
    """
    def _clean_cell(v):
        if isinstance(v, str) and ';' in v:
            parts = [p.strip() for p in v.split(';') if p.strip()]
            if len(parts) >= 2:
                last = parts[-1]
                try:
                    num = int(last)
                    penult = parts[-2].lower()
                    if num < 100 and penult in ('si', 'no', 'yes'):
                        return parts[-2]  # devolvemos penúltimo con su mayúsc/minúsc original
                    else:
                        if last in ('si', 'no', 'yes'):
                            return last
                        else:
                            return v
                except ValueError:
                    # si el último no es número, devolvemos el valor original completo
                    return v
        return v

    return df.applymap(_clean_cell)

def unify_suffix_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Para cada par de columnas base / base_<suffix>:
      - Rellena los NaN de 'base_<suffix>' con los valores de 'base'
      - Elimina luego la columna 'base'
    """
    to_drop = []
    # Iteramos sobre una copia de los nombres
    for col in list(df.columns):
        if "_" not in col:
            continue
        base, suffix = col.rsplit("_", 1)
        # si existe la columna base, unimos y marcamos para borrar la base
        if base in df.columns:
            df[col] = df[col].fillna(df[base])
            to_drop.append(base)
    return df.drop(columns=to_drop, errors="ignore")

In [ ]:
df_check = postprocess_df(df_all)

In [ ]:
import re

df_try = unify_suffix_columns(df_check)


In [ ]:
df_try.shape

In [ ]:
list_missing = list(set(df_check.columns) - set(df_try.columns))
[i for i in list_missing if 'adj' in i]

In [ ]:
list_missing[10:20]

In [ ]:
[i for i in df_all.columns if 'tipo_de_industria' in i]

In [ ]:
f = 'tipo_de_industria'
df_check[[i for i in df_check.columns if f in i]]

In [ ]:
df_try[[i for i in df_try.columns if f in i]]

In [ ]:
adjuntar_cols = [i for i in df_try.columns if 'adjuntar' in i]
[i for i in adjuntar_cols if 'ingles' in i]

In [ ]:
df_all['salida'].loc[2]